In [1]:
import os
import numpy as np
np.random.seed(251)
os.environ['PYTHONHASHSEED']=str(251)
from keras.applications.resnet50 import ResNet50, preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, GlobalAveragePooling2D, Input, concatenate
from keras.models import Sequential, Model
from keras.optimizers import SGD, Adam
from keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.utils import class_weight
from keras_efficientnets import EfficientNetB0, EfficientNet, BlockArgs

Using TensorFlow backend.


In [2]:
project_path = r'D:\UCBerkeley\CourseWork\202001\W251\Homework\Project'
dir_openpose = 'openpose_output'
dir_train = 'NEW4_image_transfer_trial4'
dir_test = 'NEW4_manual_optical_flow_output_trial4'
model_name = 'Efficientnet_model_weights_NEW4_trial4_model4.h5'

TRAIN_DIR = os.path.join(project_path, dir_openpose, dir_train)
# class_list = ['AGAIN', 'ALL', 'AWKWARD', 'BASEBALL', 'BEHAVIOR', 'CAN', 'CHAT', 'CHEAP', 
#               'CHEAT', 'CHURCH', 'COAT', 'CONFLICT', 'COURT', 'DEPOSIT', 'DEPRESS', 
#               'DOCTOR', 'DRESS', 'ENOUGH', 'NEG']
y_train_lst = [f.split('_')[1] for f in os.listdir(os.path.join(project_path, dir_openpose, dir_test)) if 'train' in f]
class_list = np.unique(y_train_lst)
sample_class_weights = class_weight.compute_class_weight('balanced', class_list, y_train_lst)

HEIGHT = 224
WIDTH = 224

In [3]:
# ResNet 50
base_model1 = ResNet50(input_shape=(HEIGHT, WIDTH, 3), include_top=True, weights='imagenet')
base_model1.summary()

# include_top=False removed two layers GlobalAveragePooling2D() and Dense()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
___________________________________________________________________________________________

In [5]:
# ResNet 50 - replace the Dense(1000) with Dense(19) but freeze all weights except for Dense(19)
base_model1 = ResNet50(input_shape=(HEIGHT, WIDTH, 3), include_top=False, weights='imagenet')
for layer in base_model1.layers:
    layer.trainable = False
base_output1 = base_model1.layers[-1].output
added1 = GlobalAveragePooling2D()(base_output1)
output_layer = Dense(19, activation='softmax')(added1)
merged_model = Model(input=base_model1.input, output=output_layer)
merged_model.summary()

C:\Users\Angela\Anaconda3\envs\w251_project3\lib\site-packages\keras_applications\resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '
C:\Users\Angela\Anaconda3\envs\w251_project3\lib\site-packages\ipykernel_launcher.py:8: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  


Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_3[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
____________________________________________________________________________________________

In [10]:
# ResNet 50 - replace the Dense(1000) with Dense(19) and retrain all weights
base_model1 = ResNet50(input_shape=(HEIGHT, WIDTH, 3), include_top=False, weights='imagenet')
base_output1 = base_model1.layers[-1].output
added1 = GlobalAveragePooling2D()(base_output1)
output_layer = Dense(19, activation='softmax')(added1)
merged_model = Model(input=base_model1.input, output=output_layer)
merged_model.summary()

Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_7[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
____________________________________________________________________________________________

C:\Users\Angela\Anaconda3\envs\w251_project3\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  


In [8]:
# EfficientNetB0 
base_model1 = EfficientNetB0(input_shape=(HEIGHT, WIDTH, 3), include_top=True, weights='imagenet')
base_model1.summary()

# include_top=False removed three layers GlobalAveragePooling2D(), Dropout(0.2) and Dense()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 112, 112, 32) 864         input_5[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 112, 112, 32) 128         conv2d_1[0][0]                   
__________________________________________________________________________________________________
swish_1 (Swish)                 (None, 112, 112, 32) 0           batch_normalization_1[0][0]      
____________________________________________________________________________________________

In [9]:
# EfficientNetB0 - replace the Dense(1000) with Dense(19) but freeze all weights except for Dense(19)
base_model1 = EfficientNetB0(input_shape=(HEIGHT, WIDTH, 3), include_top=False, weights='imagenet')
for layer in base_model1.layers:
    layer.trainable = False
base_output1 = base_model1.layers[-1].output
added1 = GlobalAveragePooling2D()(base_output1)
added1_dropout = Dropout(0.2)(added1)
output_layer = Dense(19, activation='softmax')(added1_dropout)
merged_model = Model(input=base_model1.input, output=output_layer)
merged_model.summary()

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv2d_66 (Conv2D)              (None, 112, 112, 32) 864         input_6[0][0]                    
__________________________________________________________________________________________________
batch_normalization_50 (BatchNo (None, 112, 112, 32) 128         conv2d_66[0][0]                  
__________________________________________________________________________________________________
swish_50 (Swish)                (None, 112, 112, 32) 0           batch_normalization_50[0][0]     
____________________________________________________________________________________________

C:\Users\Angela\Anaconda3\envs\w251_project3\lib\site-packages\ipykernel_launcher.py:8: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  


In [11]:
# EfficientNetB0 - replace the Dense(1000) with Dense(19) and retrain all weights
base_model1 = EfficientNetB0(input_shape=(HEIGHT, WIDTH, 3), include_top=False, weights='imagenet')
base_output1 = base_model1.layers[-1].output
added1 = GlobalAveragePooling2D()(base_output1)
added1_dropout = Dropout(0.2)(added1)
output_layer = Dense(19, activation='softmax')(added1_dropout)
merged_model = Model(input=base_model1.input, output=output_layer)
merged_model.summary()



Model: "model_7"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv2d_131 (Conv2D)             (None, 112, 112, 32) 864         input_8[0][0]                    
__________________________________________________________________________________________________
batch_normalization_99 (BatchNo (None, 112, 112, 32) 128         conv2d_131[0][0]                 
__________________________________________________________________________________________________
swish_99 (Swish)                (None, 112, 112, 32) 0           batch_normalization_99[0][0]     
____________________________________________________________________________________________

C:\Users\Angela\Anaconda3\envs\w251_project3\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  import sys
